# Week 9 (Course 2, Week 5)

In [1]:
%matplotlib inline

В этом задании будет использоваться датасет **digits** из **sklearn.datasets**. Оставьте последние **25%** объектов для контроля качества, разделив **X** и **y** на **X_train**, **y_train** и **X_test**, **y_test**.


Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами **1NN** с **RandomForestClassifier** из **sklearn** на **1000** деревьях.

## Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. __Можно не извлекать корень из суммы квадратов отклонений__, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (более строго говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeigborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со __структурами данных ball tree и kd tree__.

**Доля ошибок**, допускаемых 1NN на тестовой выборке, — **ответ в задании 1**.

In [2]:
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
import numpy as np

In [3]:
digits = load_digits()

In [4]:
X = digits.data
y = digits.target

In [5]:
X.shape, y.shape

((1797, 64), (1797,))

In [6]:
border = int(round(len(X) * 0.75))
border

1348

In [7]:
border = int(border)
border

1348

In [8]:
X_train, X_test = X[:border, :], X[border:, :]
y_train, y_test = y[:border], y[border:]

In [9]:
X_train.shape[0] + X_test.shape[0]

1797

In [10]:
def euc_metric(v1, v2):
    return np.sum((v1 - v2) ** 2)

In [11]:
def get_dist(data, target, v1):
    dist_list = []
    for i, v2 in enumerate(data):
        rec = (euc_metric(v2, v1), target[i])
        dist_list.append(rec)
    dist_list.sort()
    return dist_list

In [12]:
def knn_predict_one(data, target, test_data, test_tar):
    prediction = np.zeros(len(test_data))

    for i, v1 in enumerate(test_data):
        _, predict_y = get_dist(data, target, v1)[0]
        prediction[i] = predict_y

    return prediction

In [13]:
predictions = knn_predict_one(X_train, y_train, X_test, y_test)

In [14]:
score = accuracy_score(y_test, predictions, normalize=True)
score

0.96213808463251671

In [15]:
errors_score = 1.0 - score
errors_score

0.03786191536748329

In [16]:
with open('assignment_2_1.txt', 'wt') as f:
    f.write(str(errors_score))

In [17]:
%cat assignment_2_1.txt

0.0378619153675

## Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. **Сделайте прогнозы на тестовой выборке** и оцените долю ошибок классификации на ней. **Эта доля — ответ в задании 2**. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
estimator = RandomForestClassifier(n_estimators=1000)

In [20]:
estimator.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [21]:
predicted = estimator.predict(X_test)

In [22]:
predicted

array([7, 3, 3, 4, 6, 6, 6, 4, 9, 1, 5, 0, 9, 6, 2, 8, 2, 0, 0, 1, 7, 6, 3,
       2, 1, 7, 4, 6, 3, 1, 3, 9, 1, 7, 6, 8, 4, 3, 1, 4, 0, 5, 3, 6, 9, 6,
       1, 7, 5, 4, 4, 7, 2, 8, 2, 2, 5, 7, 9, 5, 4, 8, 8, 4, 9, 0, 8, 0, 1,
       2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4,
       5, 6, 7, 8, 9, 0, 9, 5, 5, 6, 5, 0, 9, 8, 9, 8, 4, 1, 7, 7, 3, 5, 1,
       0, 0, 2, 2, 7, 9, 2, 0, 9, 2, 6, 3, 3, 7, 3, 3, 4, 6, 6, 6, 4, 9, 9,
       5, 0, 9, 5, 2, 8, 2, 0, 0, 9, 7, 6, 3, 2, 1, 7, 4, 6, 3, 1, 3, 9, 1,
       7, 6, 8, 4, 3, 9, 4, 0, 5, 3, 6, 9, 6, 9, 7, 5, 4, 4, 7, 2, 8, 2, 2,
       5, 7, 9, 5, 4, 8, 8, 4, 9, 0, 8, 9, 8, 0, 1, 2, 3, 4, 5, 6, 7, 1, 9,
       0, 1, 2, 3, 4, 5, 6, 9, 0, 1, 2, 3, 4, 5, 6, 7, 5, 9, 4, 9, 5, 5, 6,
       5, 0, 9, 4, 5, 8, 4, 1, 7, 7, 3, 5, 1, 0, 0, 0, 2, 7, 8, 2, 0, 1, 2,
       6, 8, 7, 7, 7, 8, 4, 6, 6, 6, 7, 9, 1, 5, 0, 9, 5, 2, 8, 0, 1, 7, 6,
       3, 2, 1, 7, 7, 6, 3, 1, 3, 9, 1, 7, 6, 8, 4, 3, 1, 4, 0, 5, 3, 6, 9,
       6, 1,

In [23]:
score = accuracy_score(y_test, predicted, normalize=True)
score

0.92873051224944325

In [24]:
errors_score = 1.0 - score
errors_score

0.071269487750556748

In [25]:
with open('assignment_2_2.txt', 'wt') as f:
    f.write(str(errors_score))

In [26]:
%cat assignment_2_2.txt

0.0712694877506

In [27]:
%cat assignment_2_1.txt

0.0378619153675